In [1]:
from __future__ import print_function
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from utils import pgd_, fgsm_, gradient_information, adversarial_accuracy, gradient_norm
from Nets import MNIST_Net, Gradient_Masked_MNIST, PGD_MNIST
%load_ext autoreload
%autoreload 2
%aimport utils, Nets

## Train a NeuralNet to run experiments on

In [2]:
device = torch.device("cuda")
batch_size = 64
test_batch_size = 1000
epochs = 14
log_interval = 500

model = MNIST_Net(device=device, log_interval=log_interval, batch_size=batch_size, test_batch_size=test_batch_size)
model.train_on_data(epochs)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.332214
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.447761

Test set: Average loss: 0.0002, Accuracy: 56098/60000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.291600
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.156302

Test set: Average loss: 0.0002, Accuracy: 57132/60000 (95%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.214490
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.050031

Test set: Average loss: 0.0001, Accuracy: 57703/60000 (96%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.216857
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.129562

Test set: Average loss: 0.0001, Accuracy: 58154/60000 (97%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.178554
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.213824

Test set: Average loss: 0.0001, Accuracy: 58495/60000 (97%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.056753
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.120665

Test set: Average loss: 0.0001, Accuracy: 58714/60000 (98%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.226165
Train Epoch: 

## Attack

In [3]:
adversarial_dataset = torch.utils.data.Subset(model.test_dataset, [i for i in range(1000)])
adversarial_loader = torch.utils.data.DataLoader(adversarial_dataset, batch_size=2, num_workers=2, shuffle=False)
adversarial_accuracy(model, adversarial_loader, attack=fgsm_, eps=1)

0 / 1000
200 / 1000
400 / 1000
600 / 1000
800 / 1000
16.400000000000002


## Load Gradient Masked Network

In [4]:
masked_model = Gradient_Masked_MNIST(device=device, log_interval=log_interval, batch_size=batch_size, test_batch_size=test_batch_size)
masked_model.train_on_data(epochs)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.379560
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.775419

Test set: Average loss: 0.0005, Accuracy: 55055/60000 (92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.120899
Train Epoch: 2 [32000/60000 (53%)]	Loss: 1.143433

Test set: Average loss: 0.0003, Accuracy: 57152/60000 (95%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.156818
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.869239

Test set: Average loss: 0.0002, Accuracy: 57769/60000 (96%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.545322
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.854055

Test set: Average loss: 0.0001, Accuracy: 58161/60000 (97%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 1.018372
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.588856

Test set: Average loss: 0.0001, Accuracy: 58355/60000 (97%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 1.055253
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.886482

Test set: Average loss: 0.0001, Accuracy: 58488/60000 (97%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.428201
Train Epoch: 

## Attack

In [5]:
adversarial_dataset = torch.utils.data.Subset(masked_model.test_dataset, [i for i in range(1000)])
adversarial_loader = torch.utils.data.DataLoader(adversarial_dataset, batch_size=2, num_workers=2, shuffle=False)
adversarial_accuracy(masked_model, adversarial_loader, attack=fgsm_, eps=1)

0 / 1000
200 / 1000
400 / 1000
600 / 1000
800 / 1000
88.9


## Load PGD trained network

In [23]:
pgd_model = PGD_MNIST(device=device, log_interval=log_interval, batch_size=batch_size, test_batch_size=test_batch_size, step=0.05, eps=1, iters=20)
pgd_model.train_on_data(epochs)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.491678
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.173534

Test set: Average loss: 0.0009, Accuracy: 50384/60000 (84%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.678436
Train Epoch: 2 [32000/60000 (53%)]	Loss: 1.394930

Test set: Average loss: 0.0004, Accuracy: 55457/60000 (92%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.257905
Train Epoch: 3 [32000/60000 (53%)]	Loss: 1.185738

Test set: Average loss: 0.0003, Accuracy: 57132/60000 (95%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.072365
Train Epoch: 4 [32000/60000 (53%)]	Loss: 1.035070

Test set: Average loss: 0.0002, Accuracy: 57800/60000 (96%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.723136
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.646094

Test set: Average loss: 0.0001, Accuracy: 58186/60000 (97%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.829211
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.933726

Test set: Average loss: 0.0001, Accuracy: 58364/60000 (97%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.662220
Train Epoch: 

## Attack

In [24]:
adversarial_dataset = torch.utils.data.Subset(pgd_model.test_dataset, [i for i in range(1000)])
adversarial_loader = torch.utils.data.DataLoader(adversarial_dataset, batch_size=2, num_workers=2, shuffle=False)
adversarial_accuracy(pgd_model, adversarial_loader, attack=fgsm_, eps=1)

0 / 1000
200 / 1000
400 / 1000
600 / 1000
800 / 1000
90.9


## Gradient masking metric
Check alignment of gradient at adv point with (adv point - original point)

### Normal network

In [8]:
n_examples = 1000
x = torch.cat([model.train_dataset[i][0].unsqueeze(0) for i in range(n_examples)]).to(device)
y = torch.LongTensor([model.train_dataset[i][1] for i in range(n_examples)]).to(device)

In [9]:
[(i, np.nanmean(gradient_information(model, x, y, iters=i*10, clip_min=model.normalized_min, clip_max=model.normalized_max, device=device).detach().cpu().numpy())) for i in range(1, 40, 5)]

[(1, 0.20531067),
 (6, 0.5354743),
 (11, 0.550822),
 (16, 0.55232334),
 (21, 0.55256784),
 (26, 0.5516728),
 (31, 0.5516398),
 (36, 0.5519692)]

### FGSM trained network

In [58]:
n_examples = 1000
x = torch.cat([masked_model.train_dataset[i][0].unsqueeze(0) for i in range(n_examples)]).to(device)
y = torch.LongTensor([masked_model.train_dataset[i][1] for i in range(n_examples)]).to(device)

In [11]:
[(i, np.nanmean(gradient_information(masked_model, x, y, iters=i*10, clip_min=masked_model.normalized_min, clip_max=masked_model.normalized_max, device=device).detach().cpu().numpy())) for i in range(1, 40, 5)]

[(1, 0.18873714),
 (6, 0.48728827),
 (11, 0.5040706),
 (16, 0.50230885),
 (21, 0.5038546),
 (26, 0.5071839),
 (31, 0.5036746),
 (36, 0.49869025)]

### PGD trained network

In [25]:
n_examples = 1000
x = torch.cat([pgd_model.train_dataset[i][0].unsqueeze(0) for i in range(n_examples)]).to(device)
y = torch.LongTensor([pgd_model.train_dataset[i][1] for i in range(n_examples)]).to(device)

In [26]:
[(i, np.nanmean(gradient_information(pgd_model, x, y, iters=i*10, clip_min=pgd_model.normalized_min, clip_max=pgd_model.normalized_max, device=device).detach().cpu().numpy())) for i in range(1, 40, 5)]

[(1, 0.10904554),
 (6, 0.3140713),
 (11, 0.33384696),
 (16, 0.32671914),
 (21, 0.32991028),
 (26, 0.32651478),
 (31, 0.32870236),
 (36, 0.32701364)]

### Check gradient norms

In [27]:
n_examples = 1000
x = torch.cat([masked_model.train_dataset[i][0].unsqueeze(0) for i in range(n_examples)]).to(device)
y = torch.LongTensor([masked_model.train_dataset[i][1] for i in range(n_examples)]).to(device)

In [28]:
gradient_norm(model, x, y, device=device), gradient_norm(masked_model, x, y, device=device), gradient_norm(pgd_model, x, y, device=device)

(tensor(2.4784e-05, device='cuda:0'),
 tensor(2.4713e-05, device='cuda:0'),
 tensor(2.0038e-05, device='cuda:0'))

## Adversarial Training

In [48]:
def adv_train(model, device, train_loader, optimizer, epochs):
    model.train()
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs + 1):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            adv_data = pgd_(model, data, target, 0.1, 0.5, iters=7, targeted=False, device=device, clip_min=normalized_min, clip_max=normalized_max)
            optimizer.zero_grad()
            output = model(adv_data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

In [55]:
undefended_model = type(model)().to(device)
undefended_model.load_state_dict(model.state_dict())
adv_train(model, device, train_loader, optimizer, 4)

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.735852
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.935562
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.539442
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.542072
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.375298
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.234662
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.547802
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.331915
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.199902
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.282771
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.450014
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.228553
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.216324
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.487287
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.375047
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.198464
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.329873
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.318543
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.161249
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.292225
Train Epoch:

In [56]:
adversarial_accuracy(model, adversarial_loader)

0 / 1000
200 / 1000
400 / 1000
600 / 1000
800 / 1000
94.19999999999999


## Black Box Attack

In [29]:
def black_box_adversarial_accuracy(model, surrogate_model, dataset_loader):
    correct = 0
    for batch_idx, (data, target) in enumerate(dataset_loader):
        data, target = data.to(device), target.to(device)
        adv = fgsm_(surrogate_model, data, target, 1, targeted=False, device=device, clip_min=model.normalized_min, clip_max=model.normalized_max)
        output = model(adv)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        if (batch_idx % 100 == 0):
            print('{} / {}'.format(batch_idx * dataset_loader.batch_size, len(dataset_loader.dataset)))
    print ((correct/len(dataset_loader.dataset) * 100))

In [31]:
surrogate_model = MNIST_Net(device=device, log_interval=log_interval, batch_size=batch_size, test_batch_size=test_batch_size, oracle=masked_model)
surrogate_model.train_on_data(epochs)
adversarial_dataset = torch.utils.data.Subset(masked_model.test_dataset, [i for i in range(1000)])
adversarial_loader = torch.utils.data.DataLoader(adversarial_dataset, batch_size=2, num_workers=2, shuffle=False)
black_box_adversarial_accuracy(masked_model, surrogate_model, adversarial_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.335029
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.339845

Test set: Average loss: 0.0002, Accuracy: 55863/60000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.225276
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.206128

Test set: Average loss: 0.0002, Accuracy: 57001/60000 (95%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.188225
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.256518

Test set: Average loss: 0.0001, Accuracy: 57487/60000 (96%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.271961
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.137379

Test set: Average loss: 0.0001, Accuracy: 57873/60000 (96%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.064551
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.137387

Test set: Average loss: 0.0001, Accuracy: 58089/60000 (97%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.274478
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.066578

Test set: Average loss: 0.0001, Accuracy: 58334/60000 (97%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.165251
Train Epoch: 